In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
import lightgbm as lgb
import gc
from scipy.stats import skew, boxcox
from bayes_opt import BayesianOptimization
from scipy import sparse
from sklearn.metrics import log_loss
from datetime import datetime
%matplotlib inline

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load Data

In [2]:
train_X = pd.read_pickle('../input/' + 'train_X_2017-02-16-20-51.pkl')
test_X = pd.read_pickle('../input/' + 'test_X_2017-02-16-20-51.pkl')
test_listing = pd.read_pickle('../input/' + 'listing_id.pkl')

train_y = pd.read_pickle('../input/' + 'y_2017-02-16-20-51.pkl') 
print(train_X.shape, test_X.shape)

((49352, 229), (74659, 229))


In [3]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, train_size=.80, random_state=1234)

In [34]:
lgb.LGBMClassifier?

In [36]:
for x in [8,15,31,63,127,255]:
    rgr = lgb.LGBMClassifier(objective = 'multiclass',
                             learning_rate=0.1,
                             n_estimators=100000,
                             num_leaves=x)

    rgr.fit(X_train,y_train,
            eval_set=[(X_val,y_val)],
            eval_metric='multi_logloss',
            early_stopping_rounds=50,
            verbose = False)


    print rgr.num_leaves, ' ', rgr.evals_result_.get('valid_0').get('multi_logloss')[rgr.best_iteration]

8   0.533950251891
15   0.534017695883
31   0.533337457941
63   0.535837989584
127   0.538235118996
255   0.546729607409


In [10]:
num_leaves = 31
# 8 0.533950251891
# 15 0.534017695883
# 31 0.533337457941
# 63 0.535837989584
# 127 0.538235118996
# 255 0.546729607409

In [15]:
for x in [160,170,180,190,200,210]:
    rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
                             n_estimators=100000,
                             num_leaves=num_leaves,
                             min_child_samples = x)

    rgr.fit(X_train,y_train,
            eval_set=[(X_val,y_val)],
            eval_metric='multi_logloss',
            early_stopping_rounds=50,
            verbose = False)

    print rgr.min_child_samples, ' ', rgr.evals_result_.get('valid_0').get('multi_logloss')[rgr.best_iteration]
#     print '\n'

160   0.532305066204
170   0.533234947845
180   0.53306705402
190   0.533105412017
200   0.531902339787
210   0.534220217084


In [16]:
min_child_samples = 100
# 10   0.533337457941
# 20   0.533263247468
# 30   0.535289720244
# 40   0.533228908358
# 50   0.534171528081
# 60   0.533426694633
# 70   0.532820041162
# 80   0.532902415753
# 90   0.531844072176
# 100   0.53178303038
# 110   0.533220584562
# 120   0.533929718659
# 130   0.532662257772
# 140   0.532125059541
# 150   0.532639213339
# 160   0.532305066204
# 170   0.533234947845
# 180   0.53306705402
# 190   0.533105412017
# 200   0.531902339787
# 210   0.534220217084


In [18]:
for x in [0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
                             n_estimators=100000,
                             num_leaves=num_leaves,
                             min_child_samples = min_child_samples,
                             colsample_bytree = x)

    rgr.fit(X_train,y_train,
            eval_set=[(X_val,y_val)],
            eval_metric='multi_logloss',
            early_stopping_rounds=50,
            verbose = False)

    # print 'best_round: ', rgr.best_iteration
    print rgr.colsample_bytree, ' ', rgr.evals_result_.get('valid_0').get('multi_logloss')[rgr.best_iteration]
    

0.3   0.534473187429
0.4   0.532180375183
0.5   0.531460371729
0.6   0.52982761516
0.7   0.530897962036
0.8   0.531948878135
0.9   0.532586875189


In [19]:
colsample_bytree = 0.6

In [20]:
for x in [0.5,0.6,0.7,0.8,0.9]:
    rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
                             n_estimators=100000,
                             num_leaves=num_leaves,
                             min_child_samples = min_child_samples,
                             colsample_bytree = colsample_bytree,
                             subsample = x,
                             subsample_freq=1)

    rgr.fit(X_train,y_train,
            eval_set=[(X_val,y_val)],
            eval_metric='multi_logloss',
            early_stopping_rounds=50,
            verbose = False)

    # print 'best_round: ', rgr.best_iteration
    print rgr.subsample, ' ', rgr.evals_result_.get('valid_0').get('multi_logloss')[rgr.best_iteration]
    



0.5   0.537526883916
0.6   0.535739141308
0.7   0.533475341743
0.8   0.532631951484
0.9   0.531377832399


In [21]:
subsample = 0.9

In [23]:
for x in [15,31,63, 127, 255, 511, 1023, 2047]:
    rgr = lgb.LGBMClassifier(learning_rate=0.1,                             
                             n_estimators=100000,
                             num_leaves=num_leaves,
                             min_child_samples = min_child_samples,
                             colsample_bytree = colsample_bytree,
                             subsample = subsample,
                             subsample_freq=1,
                             max_bin = x )

    rgr.fit(X_train,y_train,
            eval_set=[(X_val,y_val)],
            eval_metric='multi_logloss',
            early_stopping_rounds=50,
            verbose = False)


    # print 'best_round: ', rgr.best_iteration
    print rgr.max_bin,' ', rgr.evals_result_.get('valid_0').get('multi_logloss')[rgr.best_iteration]
 

15   0.539288189734
31   0.535639813978
63   0.533283444874
127   0.532144055737
255   0.531377832399
511   0.531200059402
1023   0.532617672046
2047   0.532119589388


In [24]:
max_bin = 511

In [28]:
def lgbm_cv(max_bin, num_leaves, min_child_samples, colsample_bytree, subsample, learning_rate=0.1):
    skf = list(KFold(len(train_y), 5))
    scores=[]
    for i, (train, val) in enumerate(skf):
        est=lgb.LGBMClassifier(learning_rate=0.1,
                               max_bin=int(max_bin),
                               num_leaves=int(num_leaves),
                               min_child_samples=int(min_child_samples),
                               colsample_bytree=colsample_bytree,
                               subsample=subsample,
                               subsample_freq = 1
                              )
 
        train_x_fold = train_X[train]
        train_y_fold = train_y[train]
        val_x_fold = train_X[val]
        val_y_fold = train_y[val]
        est.set_params( n_estimators=100000)
        est.fit(train_x_fold,
                train_y_fold,
                eval_set=[(val_x_fold, val_y_fold)],
                eval_metric='multi_logloss',
                early_stopping_rounds=50,
                verbose = False
               )
        val_y_predict_fold = est.predict_proba(val_x_fold)
        score = log_loss(val_y_fold, val_y_predict_fold)
        scores.append(score)
    return -np.mean(scores)


lgbm_BO = BayesianOptimization(lgbm_cv, 
                               {
                                'max_bin': (200,800),
                                'num_leaves': (15,63),
                                'min_child_samples' :(80,160),
                                'colsample_bytree': (0.5,1),
                                'subsample' : (0.8,1)})

lgbm_BO.maximize(init_points=10, n_iter=40)

Initialization
-----------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_bin |   min_child_samples |   num_leaves |   subsample | 
    1 | 02m50s |   -0.53476 |             0.9494 |  206.0155 |            111.6937 |      53.0017 |      0.9958 | 
    2 | 02m11s |   -0.53134 |             0.9097 |  710.0274 |             99.8716 |      25.7253 |      0.8223 | 
    3 | 01m12s |   -0.53298 |             0.5736 |  508.9968 |            144.0163 |      28.6098 |      0.9377 | 
    4 | 01m24s |   -0.53490 |             0.8441 |  683.6996 |             99.5012 |      60.9060 |      0.9267 | 
    5 | 01m06s |   -0.53391 |             0.5226 |  508.3731 |             85.1011 |      36.6520 |      0.8504 | 
    6 | 01m02s |   -0.53513 |             0.6736 |  319.5854 |            124.2871 |      61.2123 |      0.8560 | 
    7 | 01m09s |   -0.53263 |             0.6901 |  470.4126 |    

/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
/home/xujin/AI/anaconda2/lib/python2.7/site-packages/bayes_opt/helpers.py:95: RuntimeWarning: divide by zero encountered in true_divide
  z = (mean - y_max - xi)/std


Bayesian Optimization
-----------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_bin |   min_child_samples |   num_leaves |   subsample | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00035908]), 'nit': 5, 'funcalls': 51}
  " state: %s" % convergence_dict)


   11 | 01m42s |   -0.53312 |             0.5305 |  797.2248 |             92.6381 |      25.8138 |      0.8091 | 


...

   46 | 01m55s |   -0.53223 |             0.7986 |  790.9825 |            139.1079 |      17.6317 |      0.8300 | 
   47 | 01m38s |   -0.53181 |             0.5268 |  468.2110 |            100.4549 |      20.0324 |      0.8654 | 
   48 | 01m55s |   -0.53173 |             0.7773 |  460.9678 |            157.3346 |      15.2557 |      0.9048 | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.0005743]), 'nit': 6, 'funcalls': 49}
  " state: %s" % convergence_dict)


   49 | 01m42s |   -0.53184 |             0.5113 |  260.9516 |            113.5624 |      15.8408 |      0.8203 | 


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([ 0.00112517]), 'nit': 3, 'funcalls': 46}
  " state: %s" % convergence_dict)


   50 | 01m40s |   -0.53239 |             0.7990 |  340.5197 |            105.1115 |      28.4028 |      0.9560 | 


In [29]:
def lgbm_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    print ("Blend %d estimators for %d folds" % (len(estimators), fold))
    skf = list(KFold(len(train_y), fold))
    N_class = len(set(train_y))
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*len(estimators)))
    test_blend_x = np.zeros((test_x.shape[0], N_class*len(estimators)))
    scores = np.zeros ((len(skf),len(estimators)))
    best_rounds = np.zeros ((len(skf),len(estimators)))
    

    
    for j, est in enumerate(estimators):
        print ("Model %d: %s" %(j+1, est))
        test_blend_x_j = np.zeros((test_x.shape[0], N_class*len(skf)))
        for i, (train, val) in enumerate(skf):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train]
            train_y_fold = train_y[train]
            val_x_fold = train_x[val]
            val_y_fold = train_y[val]
            if early_stopping_rounds==0: # without early stopping
                est.fit(train_x_fold, train_y_fold)
                best_rounds[i,j]=est.n_estimators
                val_y_predict_fold = est.predict_proba(val_x_fold)
                score = log_loss(val_y_fold, val_y_predict_fold)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, (j*N_class):(j+1)*N_class] = val_y_predict_fold
                test_blend_x_j[:,i] = est.predict_proba(test_x)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))
            else:                        # early stopping
                est.set_params( n_estimators=100000)
                est.fit(train_x_fold,
                        train_y_fold,
                        eval_set=[(val_x_fold, val_y_fold)],
                        eval_metric='multi_logloss',
                        early_stopping_rounds=early_stopping_rounds,
                        verbose=False
                       )
                best_round=est.best_iteration
                best_rounds[i,j]=best_round
                print ("best round %d" % (best_round))
                val_y_predict_fold = est.predict_proba(val_x_fold,num_iteration=best_round)
                score = log_loss(val_y_fold, val_y_predict_fold)
                print ("Score: ", score)
                scores[i,j]=score
                train_blend_x[val, (j*N_class):(j+1)*N_class] = val_y_predict_fold
                test_blend_x_j[:,(i*N_class):(i+1)*N_class] = est.predict_proba(test_x,num_iteration=best_round)
                print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
       
        test_blend_x_j = pd.DataFrame(test_blend_x_j)
        for N in range(N_class):
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds )

In [30]:
gbm_bo_scores = pd.DataFrame([[s[0]['num_leaves'],
                               s[0]['min_child_samples'],
                               s[0]['max_bin'],
                               s[0]['colsample_bytree'],
                               s[0]['subsample'],
                               s[1]] for s in zip(lgbm_BO.res['all']['params'],lgbm_BO.res['all']['values'])],
                            columns = ['num_leaves',
                                       'min_child_samples',
                                       'max_bin',
                                       'colsample_bytree',
                                       'subsample',
                                       'score'])
gbm_bo_scores=gbm_bo_scores.sort_values('score',ascending=False)
gbm_bo_scores.head()

,num_leaves,min_child_samples,max_bin,colsample_bytree,subsample,score
20,16.458039,117.622103,478.697403,0.842639,0.820588,-0.531018
1,15.313144,116.494344,290.673175,0.762613,0.843922,-0.531225
13,16.691140,80.808810,202.206598,0.870567,0.861266,-0.531231
10,18.654455,158.680310,327.356657,0.754814,0.930391,-0.531267
9,18.192282,124.665623,333.804592,0.849373,0.912663,-0.531491


In [37]:
estimators = [lgb.LGBMClassifier(
                     learning_rate=0.01, ## use smaller learning rate for better accuracies
                     n_estimators=100000,
                     max_bin=478,
                     num_leaves=16,
                     min_child_samples=117,
                     colsample_bytree=0.842639,
                     subsample=0.820588,
                     subsample_freq=1),
#               score -0.531018
              lgb.LGBMClassifier(
                     learning_rate=0.01, ## use smaller learning rate for better accuracies
                     n_estimators=100000,
                     max_bin=290,
                     num_leaves=15,
                     min_child_samples=116,
                     colsample_bytree=0.762613,
                     subsample=0.843922,
                     subsample_freq=1),
#               score -0.531225
              lgb.LGBMClassifier(
                     learning_rate=0.01, ## use smaller learning rate for better accuracies
                     n_estimators=100000,
                     max_bin=202,
                     num_leaves=16,
                     min_child_samples=80,
                     colsample_bytree=0.870567,
                     subsample=0.861266,
                     subsample_freq=1),
#               score -0.531231
              lgb.LGBMClassifier(
                     learning_rate=0.01, ## use smaller learning rate for better accuracies
                     n_estimators=100000,
                     max_bin=327,
                     num_leaves=18,
                     min_child_samples=158,
                     colsample_bytree=0.754814,
                     subsample=0.930391,
                     subsample_freq=1),
#               score -0.531267
              lgb.LGBMClassifier(
                     learning_rate=0.01, ## use smaller learning rate for better accuracies
                     n_estimators=100000,
                     max_bin=333,
                     num_leaves=18,
                     min_child_samples=124,
                     colsample_bytree=0.849373,
                     subsample=0.912663,
                     subsample_freq=1),
#               score -0.531491              
        ]

(train_blend_x_gbm,
 test_blend_x_gbm,
 blend_scores_gbm,
 best_rounds_gbm) = lgbm_blend(estimators, train_X, train_y, test_X,
                                 5,
                                 500) #as the learning rate decreases the number of stopping rounds need to be increased



Blend 5 estimators for 5 folds
Model 1: LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.842639, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.01, max_bin=478, max_depth=-1,
        max_drop=50, min_child_samples=117, min_child_weight=5,
        min_split_gain=0, n_estimators=100000, nthread=-1, num_leaves=16,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5,
        subsample=0.820588, subsample_for_bin=50000, subsample_freq=1,
        uniform_drop=False, xgboost_dart_mode=False)
Model 1 fold 1
best round 5753
('Score: ', 0.53784225704008859)
Model 1 fold 1 fitting finished in 274.031s
Model 1 fold 2
best round 5187
('Score: ', 0.52171515148042436)
Model 1 fold 2 fitting finished in 241.103s
Model 1 fold 3
best round 4922
('Score: ', 0.52046110692644332)
Model 1 fold 3 fitting finished in 237.563s
Model 1 fold 4
best round 4465
('Score: ', 0.5278600588979343)
Model 1 fold 4 fitting fi

In [38]:

now = datetime.now()

name_train_blend = '../output/train_blend_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores_gbm,axis=0))
print (np.mean(best_rounds_gbm,axis=0))
np.savetxt(name_train_blend,train_blend_x_gbm, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_gbm, delimiter=",")

[ 0.52918957  0.52911925  0.52882423  0.52958864  0.52949003]
[ 5081.6  5483.8  5253.4  4707.8  4640.8]


In [45]:
now = datetime.now()
df = pd.read_json(open("../input/test.json", "r"))
labels2idx ={'high': 0, 'low': 2, 'medium': 1}
sub_name = '../output/sub_LightGBM_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

sub = pd.DataFrame()
sub["listing_id"] = df["listing_id"]

y_test = np.zeros((df.shape[0], 3))

for N in range(3):
    y_test[:,N] = pd.DataFrame(test_blend_x_gbm).iloc[:,[x for x in range(test_blend_x_gbm.shape[1]) if x%3 == N]].mean(axis=1)
    
for label in ["high", "medium", "low"]:
    sub[label] = y_test[:, labels2idx[label]]
sub.to_csv(sub_name, index=False)